In [37]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import networkx as nx

from tensorflow.examples.tutorials.mnist import input_data

from tools import combination_policy
from tools.show_tf_graph import show_graph
from copy import deepcopy

In [2]:
mnist = input_data.read_data_sets('/data/mnist', one_hot=True)

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [3]:
BATCH_SIZE = 32
N_EPOCHS = 30
N_AGENTS = 3
A = np.ones((N_AGENTS,N_AGENTS))/N_AGENTS

## Early Development

### Topology

In [20]:
class Topology:
    
    def __init__(self, n_agents=5, density=0.6, policy='metropolis', adjacency_matrix=None):
        if adjacency_matrix is not None:
            assert(adjacency_matrix.shape[0] == adjacency_matrix.shape[1])
            self.adjacency_matrix = adjacency_matrix
            self.n_agents = adjacency_matrix.shape[0]
        else:
            graph = nx.random_geometric_graph(n_agents, radius=density)
            if policy == 'metropolis':
                adjacency_matrix = combination_policy.metropolis_matrix(graph)
            elif policy == 'average':
                adjacency_matrix = combination_policy.averaging_matrix(graph)             
            else:
                print('Combination policy {0} not supported yet.'.format(rule))
                raise NotImplementedError
            self.adjacency_matrix = adjacency_matrix
            self.n_agents = n_agents
        
    def neighbor_weights(self, node_index):
        return self.adjacency_matrix[node_index, :]
    

In [21]:
t = Topology()

In [23]:
t.neighbor_weights(1)

array([ 0.,  1.,  0.,  0.,  0.])

### Model

In [40]:
class Model:
    '''
    self.agent_index
    self.data
    self.parameters
    self.loss
    
    get_gradients
    gradient_descent
    '''
    
    def __init__(self, agent_index, data_dict, parameters_dict, loss_model='logistic'):
        '''
        data_dict: a dict of the form {placeholder_name: tf.placeholder object}
        parameters_dict: a dict of the form {variable_name: tf.Variable object}
        '''
        self.data = dict()
        self.parameters = dict()
        suffix = str(agent_index)
        with tf.name_scope('model' + suffix):
            with tf.name_scope('data' + suffix):
                for name, data_ph in data_dict.items():
                    self.data[name] = tf.identity(data_ph)

            with tf.name_scope('parameters' + suffix):
                for name, param in parameters_dict.items():
                    self.parameters[name] = tf.identity(param)

            with tf.name_scope('loss' + suffix):
                if loss_model == 'logistic':
                    # logistic model using softmax
                    # data_dict = {'X': predictor, 'y': predicted_value}
                    # parameters_dict = {'W': coefficient, 'b': bias}
                    # loss = cross_entropy(softmax(XW + b), y)
                    self.loss = tf.reduce_mean(
                        tf.nn.softmax_cross_entropy_with_logits(
                        logits=tf.matmul(self.data['X'], self.parameters['W']) + self.parameters['b'], 
                        labels=self.data['y'], 
                        name='loss'))
                else: 
                    print('Loss model {0} not supported yet.'.format(loss_model))
                    raise NotImplementedError
    
    def get_gradients(self, parameter_names=None):
        '''
        returns gradients_dict {param_name: gradient_of_param_wrt_loss}
        '''
        if parameter_names is None:
            parameter_names = self.parameters.keys()
        gradients = tf.gradients(loss, [self.parameters[name] for name in parameter_names])
        return dict(zip(parameter_names, gradients))
    
    def descent_update(self, step_size, parameter_names=None, gradients_dict=None):
        '''
        returns descent_dict that updates parameters in parameter_names by -step_size*gradient
        '''
        if parameter_names is None:
            parameter_names = self.parameter.keys()
            gradients_dict = self.get_gradients()
        descent_ops = dict()
        for name in parameter_names:
            descent[name] = tf.assign_sub(self.parameters[name], step_size * gradients_dict[name])
        return descent_dict

### Agent

In [ ]:
class Agent:
    '''
    
    '''
    def __init__(self, data, model, scheme, neighbor_weights):
        self.data = data  # tf Dataset object
        self.model = model
        self.scheme = scheme
        self.neighbor_weights = neighbor_weights
        
    def train(sess, n_steps, step_size):
        # initialization
        
        # computation phase
        
        # communication phase
        
        

### Cluster

In [ ]:
class Cluster:
    
    def __init__(self, dataset, model, topology=Topology()):
        self.dataset = dataset
        self.model = model
        self.topology = topology
        self.agents = [self.generate_agent(agent_index) 
                       for agent_index in range(self.topology.n_agents)]
    
    def generate_agent(self, agent_index):
        return Agent(self.model, )
        
        
    def distribute_data(self):
        
    

### Test

In [43]:
tf.reset_default_graph()

with tf.name_scope('distributor'):
    W = tf.Variable(tf.zeros([784, 10]), dtype=tf.float32, name='W')
    b = tf.Variable(tf.zeros([1, 10]), dtype=tf.float32, name='b')
    X = tf.placeholder(shape=[32, 784], dtype=tf.float32, name='X')
    y = tf.placeholder(shape=[32, 10], dtype=tf.int16, name='y')


data = {'X': X, 'y': y}
param = {'W': W, 'b': b}

m1 = Model(1, data, param)
m2 = Model(2, data, param)
show_graph(tf.get_default_graph())

In [7]:
class Agent:
    
    BATCH_SIZE = 32
    N_EPOCHS = 30
    
    def __init__(self):
        with tf.name_scope('par'):
            self.w = tf.Variable(tf.zeros([784, 10]), dtype=tf.float32, name='w')
            self.b = tf.Variable(tf.zeros([1, 10]), dtype=tf.float32, name='b')
        with tf.name_scope('data'):
            self.X = tf.placeholder(shape=[BATCH_SIZE, 784], dtype=tf.float32, name='image')
            self.Y = tf.placeholder(shape=[BATCH_SIZE, 10], dtype=tf.int16, name='label')
        with tf.name_scope('loss'):
            self.logits = tf.matmul(self.X, self.w) + self.b
            self.entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y, name='loss')
            self.loss = tf.reduce_mean(self.entropy)
        with tf.name_scope('opt'):
            self.optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(self.loss)
            
    def train(self, mnist):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            N_BATCHES = int(mnist.train.num_examples/BATCH_SIZE)
            for i in range(N_EPOCHS):
                total_loss = 0
                for _ in range(N_BATCHES):
                    X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
                    _, loss_batch = sess.run([self.optimizer, self.loss], feed_dict={self.X: X_batch, self.Y: Y_batch})
                    total_loss += loss_batch
                print('Average loss epoch {0}: {1}'.format(i, total_loss/N_BATCHES))
            print('Optimization Finished!')

In [8]:
a1 = Agent()
a2 = Agent()

In [10]:
a = Agent()
a.train(mnist)

Average loss epoch 0: 0.7912245929171792
Average loss epoch 1: 0.4651583866828812
Average loss epoch 2: 0.41057026798254675
Average loss epoch 3: 0.38279259577827207
Average loss epoch 4: 0.3659034872490466
Average loss epoch 5: 0.35371557174291685
Average loss epoch 6: 0.34420496931384137
Average loss epoch 7: 0.3366953860447938
Average loss epoch 8: 0.33073767690081257
Average loss epoch 9: 0.32582427109607925
Average loss epoch 10: 0.3213129277136155
Average loss epoch 11: 0.31764159218014604
Average loss epoch 12: 0.3136022969477564
Average loss epoch 13: 0.31147762451741556
Average loss epoch 14: 0.30839795580435203
Average loss epoch 15: 0.3058674128268595
Average loss epoch 16: 0.3045638434185589
Average loss epoch 17: 0.3018722187953202
Average loss epoch 18: 0.29968145338772895
Average loss epoch 19: 0.29827701309706683
Average loss epoch 20: 0.29690889467521026
Average loss epoch 21: 0.2953184770964459
Average loss epoch 22: 0.2933997828903694
Average loss epoch 23: 0.2926741